In [1]:
import json
import requests
import pandas as pd
import time
from datetime import datetime
import psycopg2

import import_ipynb
import tcg_bearer_token

from psycopg2.extras import execute_values

importing Jupyter notebook from tcg_bearer_token.ipynb


In [2]:
def get_data_from_ids(productIDs, token):
    results = []
    tot_len = len(productIDs)
    print(tot_len)
    loops = 0
    while loops * 250 < tot_len:
        converted_ids = []
        for item in productIDs[loops * 250:(loops + 1) * 250]:
            try:
                converted_ids.append(str(int(item)))
            except ValueError as e:
                print('Nan value')

        url = "https://api.tcgplayer.com/pricing/product/" + ",".join(converted_ids)

        headers = {'accept': 'application/json', 'authorization': 'bearer ' + token}

        response = requests.get(url, headers=headers)

        results += response.json()['results']
        loops += 1
        print(loops)
        time.sleep(.1)
    return results

In [3]:
def pull_tcg_data():
    conn = psycopg2.connect(host="localhost", port=5432, database="", user="", password="")

    SQL = 'SELECT uuid, cardname, setname, tcgplayerproductid FROM cardinfo;'
    all_cards = pd.read_sql_query(sql=SQL, con=conn)
    
    conn.close()

    token = tcg_bearer_token.get_bearer()
    prices = get_data_from_ids(all_cards['tcgplayerproductid'], token)
    pre_df = []
    for item in prices:
        id = item['productId']
        low = item['lowPrice']
        mid = item['midPrice']
        high = item['highPrice']
        market = item['marketPrice']
        direct = item['directLowPrice']
        if item['subTypeName'] == 'Foil':
            foil = True
        else:
            foil = False
        pre_df.append([id, foil, low, mid, high, market, direct])

    data_df = pd.DataFrame(pre_df, columns=['productId', 'foil', 'lowPrice', 'midPrice', 'highPrice', 'marketPrice',
                                            'directLowPrice'])
    data_df.dropna(thresh=3, inplace=True)
    data_df['tcgDate'] = datetime.now()

    conn = psycopg2.connect(host="localhost", port=5432, database="", user="", password="")
    cur = conn.cursor()
    
    data_df.drop_duplicates(subset=['productId', 'foil', 'tcgDate'], inplace=True)
    data_df.fillna(value=0, inplace=True)
    
    t_df = tuple(data_df[['productId', 'foil', 'tcgDate', 'lowPrice', 'midPrice', 'highPrice', 'marketPrice', 
                    'directLowPrice']].itertuples(index=False, name=None))
    execute_values(cur, "INSERT INTO tcgpricedata (tcgplayerproductid, foil, tcgdate, lowprice, midprice, highprice, marketprice, directlowprice) VALUES %s", t_df)
    conn.commit()

    cur.close()
    conn.close()

In [4]:
def main():
    pull_tcg_data()

In [5]:
if __name__ == '__main__':
    main()

46014
1
Nan value
2
Nan value
3
4
5
Nan value
Nan value
6
Nan value
Nan value
7
Nan value
8
9
10
11
Nan value
12
Nan value
13
14
Nan value
15
Nan value
16
17
Nan value
Nan value
18
19
20
21
22
Nan value
23
24
Nan value
25
Nan value
Nan value
26
27
28
29
Nan value
30
31
32
Nan value
33
34
Nan value
Nan value
Nan value
Nan value
35
Nan value
36
37
38
39
40
41
42
43
44
Nan value
Nan value
Nan value
45
46
47
48
49
Nan value
50
51
Nan value
52
53
Nan value
54
Nan value
55
Nan value
Nan value
56
Nan value
Nan value
Nan value
57
58
59
Nan value
60
Nan value
61
62
Nan value
63
64
65
66
Nan value
67
68
69
70
71
72
Nan value
73
74
75
76
Nan value
77
78
79
80
81
82
83
84
85
86
87
88
89
90
Nan value
91
92
93
94
Nan value
95
Nan value
Nan value
96
Nan value
97
Nan value
98
Nan value
Nan value
99
100
Nan value
101
Nan value
102
Nan value
103
Nan value
Nan value
Nan value
Nan value
Nan value
Nan value
104
Nan value
Nan value
Nan value
Nan value
Nan value
105
106
107
108
109
110
111
112
Nan value
113
